In [111]:

# sourcery skip: for-index-underscore
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import json

In [112]:
url = "https://www.tmz.com"
req = requests.get(url)
doc = BeautifulSoup(req.text)

In [113]:
raw_titles = doc.select("header > a > h2")

In [114]:
# Grabs all the headlines, appends period if doesn't end in !
count = 0
headlines = []
valid_punct = ["'", "!", "?"]
for title in tqdm(enumerate(raw_titles)):
    print("- - -")
    headline = raw_titles[count].get_text().strip().replace("\n", " ") + "."
    if headline[-2] in valid_punct:
        headline = headline[:-1]
    else: pass

    print(headline)
    headlines.append(headline)
    count += 1

16it [00:00, 80756.76it/s]

- - -
Vin Scully Dodgers Broadcasting Legend ... Dead At 94.
- - -
Jon Stewart Vet Bill Pressure Works ... GOP Senators Flip, Pass PACT Act.
- - -
Willow Smith Stops Show After Fan Faints in Crowd.
- - -
LeBron James Hits Lakers Facility W/ Sons ... Epic Workout With My Boys!!!
- - -
'Ferris Bueller' Star Edie McClurg Allegedly Elder Abuse Victim ... Caregiver Allegedly Sexually Assaulted.
- - -
Killer Mike ATL's Black Biz Owners in Danger ... I'm Taking a Stand!!!
- - -
Demi Lovato Adds She/Her Pronouns to They/Them ... 'I've Been Feeling More Feminine Recently'
- - -
Kanye West Rips Adidas For 'Yeezy Day' ... I Didn't Approve!!!
- - -
Thomas Markle Tabloid Honcho Files for Restraining Order ... He's Gonna Kill Me!!!
- - -
Deshaun Watson Ban NFL In 'Really Tricky' Appeal Situation ... Andrew Brandt Says.
- - -
Soccer Player Attacks Female Ref Over Yellow Card ... Arrested.
- - -
'Twilight' Star Cam Gigandet Wife Files For Divorce.
- - -
Paraglider Narrowly Cheats Death ... Terrifying 

In [115]:
def get_tags(article, position):
    tags = doc.select("section.tag-cloud > ul > a")
    tag_list = []
    for tag in tags:
        data = json.loads(tag['data-context'])
        if data['pos'] == position:
            tag_list.append(tag.text.strip())
    return tag_list

In [137]:
def break_timestamp(article, position):
    timestamps = doc.select(".article")
        # grab, clean timestamp
    timestamp = timestamps[position].text.split("PT")[-20:]
    timestamp = timestamp[0][-20:]
    timestamp = timestamp.strip()
    # timestamp = datetime.st
    calendar = timestamp[:9]
    clock =  timestamp[-8:].strip()
    
    date_a_frame = pd.DataFrame()
    # date_a_frame["calendar"] = datetime.strptime(calendar, "%b %d %Y %I:%M%p")
    # date_a_frame["clock"] = clock
    year = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").year
    month = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").month
    day = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").day
    hour = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").hour
    minute = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").minute

    datetimes = {
        "calendar": calendar,
        "clock": clock,
        "year": year,
        "month": month,
        "day": day,
        "hour": hour,
        "minute": minute
    }
    meta = []
    meta.append(datetimes)
    return datetimes

In [139]:
break_timestamp(raw_titles, 2)

{'calendar': '8/2/2022 ',
 'clock': '6:12 PM',
 'year': 2022,
 'month': 8,
 'day': 2,
 'hour': 18,
 'minute': 12}

In [142]:
# init dataframe of tags
tags = doc.select("section.tag-cloud > ul > a")

valid_punct = ["'", "!", "?"]

dicts = []

count = 0
for article_position, article in enumerate(raw_titles, start=1):
    headline = raw_titles[count].get_text().strip().replace("\n", " ") + "."
    if headline[-2] in valid_punct:
        headline = headline[:-1]
    tag_list = get_tags(article, count + 1)

    datetimes = break_timestamp(article, count)

    article_dict = {
        "headline": headline,
        "tags": tag_list,
        "timestamp": timestamp,
        "calendar": datetimes["calendar"],
        "clock": datetimes["clock"],
        "year": datetimes["year"],
        "month": datetimes["month"],
        "day": datetimes["day"],
        "hour": datetimes["hour"],
        "minute": datetimes["minute"],
    }
    dicts.append(article_dict)
    count += 1
test = pd.DataFrame(dicts)
test

,headline,tags,timestamp,calendar,clock,year,month,day,hour,minute
0,Vin Scully Dodgers Broadcasting Legend ... Dea...,"[Vin Scully, TMZ Sports]",8/2/2022 6:22 PM,8/2/2022,8:30 PM,2022,8,2,20,30
1,Jon Stewart Vet Bill Pressure Works ... GOP Se...,"[Jon Stewart, Ted Cruz, You Might Want to Reth...",8/2/2022 6:22 PM,8/2/2022,6:22 PM,2022,8,2,18,22
2,Willow Smith Stops Show After Fan Faints in Cr...,"[Willow Smith, Machine Gun Kelly, Nurse!, Cont...",8/2/2022 6:22 PM,8/2/2022,6:12 PM,2022,8,2,18,12
3,LeBron James Hits Lakers Facility W/ Sons ... ...,"[LeBron James, Lebron James jr, Bryce James, T...",8/2/2022 6:22 PM,8/2/2022,2:57 PM,2022,8,2,14,57
4,'Ferris Bueller' Star Edie McClurg Allegedly E...,"[Edie McClurg, Movies, Sex, Nurse!, Exclusive,...",8/2/2022 6:22 PM,8/2/2022,1:41 PM,2022,8,2,13,41
5,Killer Mike ATL's Black Biz Owners in Danger ....,"[Killer Mike, Hip Hop, Restaurants & Nightclub...",8/2/2022 6:22 PM,8/2/2022,2:57 PM,2022,8,2,14,57
6,Demi Lovato Adds She/Her Pronouns to They/Them...,"[Demi Lovato, LGBT, Breaking News, Viral, Music]",8/2/2022 6:22 PM,8/2/2022,1:09 PM,2022,8,2,13,9
7,Kanye West Rips Adidas For 'Yeezy Day' ... I D...,"[Adidas, Kanye West, Money, Hip Hop, Fashion, ...",8/2/2022 6:22 PM,8/2/2022,1:12 PM,2022,8,2,13,12
8,Thomas Markle Tabloid Honcho Files for Restrai...,"[Meghan Markle, Prince Harry, Family, Exclusiv...",8/2/2022 6:22 PM,8/2/2022,11:47 AM,2022,8,2,11,47
9,Deshaun Watson Ban NFL In 'Really Tricky' Appe...,"[Deshaun Watson, Cleveland Browns, Houston Tex...",8/2/2022 6:22 PM,8/2/2022,12:17 PM,2022,8,2,12,17


In [40]:
temp

,calendar,clock,year


In [85]:
# init var timestamp
# sourcery skip: for-index-underscore
tmz_timestamps = doc.select(".article")
stories = pd.DataFrame()
story_count = 0
for trash in enumerate(tmz_timestamps):

    # grab, clean timestamp
    timestamp = tmz_timestamps[story_count].text.split("PT")[-20:]
    timestamp = timestamp[0][-20:]
    timestamp = timestamp.strip()

    # set timestamp, date, time, headline
    story = {
        "timestamp": timestamp,
        "date": timestamp[:9],
        "time": timestamp[-8:].strip(),
        "headline": headline,
    }

    # convert to df
    story = pd.DataFrame([story])
        # convert story 12AP to 24
    try:
        story["time"] = pd.to_datetime(story["time"], format="%I:%M %p").dt.strftime(
            "%H:%M"
        )
    except Exception:
        story["time"] = story["time"]
    
    story["tags"] = get_tags(trash, story_count)

    story.convert_dtypes()

    # Parsing the date and time into columns.
    story["year"] = pd.DatetimeIndex(story["date"]).year
    story["month"] = pd.DatetimeIndex(story["date"]).month
    story["day"] = pd.DatetimeIndex(story["date"]).day
    story["hour"] = pd.DatetimeIndex(story["time"]).hour
    story["minute"] = pd.DatetimeIndex(story["time"]).minute

    stories = pd.concat([stories, story], ignore_index=True)
    
    story_count += 1
# Creating a new dataframe with the columns headline, timestamp, year, month, day,
# hour, and minute.

    
stories = stories[["headline", "tags", "timestamp", "year", "month", "day", "hour", "minute"]]


# This is reading the headlines.csv file and converting it to a dataframe.
existing = pd.read_csv("../headlines.csv")
existing = existing.dropna(axis=1).set_index("Unnamed: 0")
existing.convert_dtypes()

# Parsing the timestamp into columns.
existing["year"] = pd.DatetimeIndex(existing["timestamp"]).year
existing["month"] = pd.DatetimeIndex(existing["timestamp"]).month
existing["day"] = pd.DatetimeIndex(existing["timestamp"]).day
existing["hour"] = pd.DatetimeIndex(existing["timestamp"]).hour
existing["minute"] = pd.DatetimeIndex(existing["timestamp"]).minute

# This is a try/except block. It is trying to read the headlines.csv file and
# convert it to a dataframe with time-parsed columns. If it fails, it will pass.
try:
    existing = pd.read_csv("/Users/bean/Dropbox/code/03_projects/tmz-poetry/headlines.csv")
    existing["year"] = pd.DatetimeIndex(existing["timestamp"]).year
    existing["month"] = pd.DatetimeIndex(existing["timestamp"]).month
    existing["day"] = pd.DatetimeIndex(existing["timestamp"]).day
    existing["hour"] = pd.DatetimeIndex(existing["timestamp"]).hour
    existing["minute"] = pd.DatetimeIndex(existing["timestamp"]).minute
    existing = existing[
        [
            "headline",
            "tags",
            "timestamp",
            "date",
            "time",
            "year",
            "month",
            "day",
            "hour",
            "minute",
        ]
    ]
except Exception:
    pass

# This is concatenating the existing dataframe with the new dataframe.
export = pd.concat([existing, stories], ignore_index=True)
export = export[["headline", "tags", "timestamp", "year", "month", "day", "hour", "minute"]]

# Dropping duplicates and sorting the dataframe by year, month, day, hour, and
# minute.
export = export.drop_duplicates(subset=["timestamp"], keep="last")
export = export.sort_values(
    by=["year", "month", "day", "hour", "minute"],
    ascending=[False, False, False, False, False],
    ignore_index=True,
)
export

ValueError: Length of values (0) does not match length of index (1)